Load Library


In [ ]:
from keras.applications.xception import Xception
from keras.preprocessing import image
import numpy as np
from keras.layers import Dense, Dropout
from keras.models import Model
import os
from keras.layers import Input,Conv2D,BatchNormalization,concatenate,Dense,Dropout,GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
from keras.metrics import categorical_accuracy
from keras.callbacks import Callback, ModelCheckpoint,LearningRateScheduler,EarlyStopping,ReduceLROnPlateau
from keras.optimizers import SGD,Adam
from keras import regularizers

In [ ]:
import torch
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

Load Driver

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load data set(for single subject)

In [ ]:
def Load_data(s,fea):
  X_train=[]
  Y_train=[]
  X_test=[]
  Y_test=[]
  # s_weight='/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/'+str(fea)+'_weight.h5'
  
  path='/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/s'+str(s)+'/'+str(fea)
    
  train=path+'/train'
  test=path+'/test'
  for g in range(1,53):
    for t in [0,2,3,5,6,7,8]:
      x='s'+str(s)+'g'+str(g)+'tr'+str(t)+'.png'
      f=train+'/'+x
      img=image.load_img(f)
      img=image.img_to_array(img)
      img=image.smart_resize(img, (100,200), interpolation='bilinear')
        
      if x[-9]!='g':

        label=x[-9:-7]
      else:
        label=x[-8:-7]

      X_train.append(img)
      Y_train.append(label)
  for g in range(1,53):
    for t in [1,4,9]:
      x='s'+str(s)+'g'+str(g)+'tr'+str(t)+'.png'
      f=test+'/'+x
      img=image.load_img(f)
      img=image.img_to_array(img)
      img=image.smart_resize(img, (100,200), interpolation='bilinear')
      if x[-9]!='g':
        label=x[-9:-7]
      else:
        label=x[-8:-7]

      X_test.append(img)
      Y_test.append(label)
  
  X_train=np.asarray(X_train,np.float32)
  X_test=np.asarray(X_test,np.float32)
  Y_train=np.asarray(Y_train,np.float32)
  Y_test=np.asarray(Y_test,np.float32)
  print('shape of training data:',X_train.shape)
  print('shape of training label:',Y_train.shape)
  print('shape of test data:',X_test.shape)
  print('shape of test label:',Y_test.shape)
  
  return X_train,Y_train,X_test,Y_test



In [ ]:
def Load_data_all(fea):
  X_train=[]
  Y_train=[]
  X_test=[]
  Y_test=[]
  for s in range(1,28):
    path='/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/s'+str(s)+'/'+str(fea)
    
    train=path+'/train'
    test=path+'/test'
    for g in range(1,53):
      for t in [0,2,3,5,6,7,8]:
        x='s'+str(s)+'g'+str(g)+'tr'+str(t)+'.png'
        f=train+'/'+x
        img=image.load_img(f)
        img=image.img_to_array(img)
        img=image.smart_resize(img, (100,200), interpolation='bilinear')
          
        if x[-9]!='g':

          label=x[-9:-7]
        else:
          label=x[-8:-7]

        X_train.append(img)
        Y_train.append(label)
    for g in range(1,53):
      for t in [1,4,9]:
        x='s'+str(s)+'g'+str(g)+'tr'+str(t)+'.png'
        f=test+'/'+x
        img=image.load_img(f)
        img=image.img_to_array(img)
        img=image.smart_resize(img, (100,200), interpolation='bilinear')
        if x[-9]!='g':
          label=x[-9:-7]
        else:
          label=x[-8:-7]

        X_test.append(img)
        Y_test.append(label)
  X_train=np.asarray(X_train,np.float32)
  X_test=np.asarray(X_test,np.float32)
  Y_train=np.asarray(Y_train,np.float32)
  Y_test=np.asarray(Y_test,np.float32)
  print('shape of training data:',X_train.shape)
  print('shape of training label:',Y_train.shape)
  print('shape of test data:',X_test.shape)
  print('shape of test label:',Y_test.shape)
  
  return X_train,Y_train,X_test,Y_test



In [ ]:
X_train_iemg,Y_train_iemg,X_test_iemg,Y_test_iemg=Load_data_all('IEMG')


shape of training data: (9828, 100, 200, 3)
shape of training label: (9828,)
shape of test data: (4212, 100, 200, 3)
shape of test label: (4212,)


In [ ]:
X_train_mav,Y_train_mav,X_test_mav,Y_test_mav=Load_data_all('MAV')

shape of training data: (9828, 100, 200, 3)
shape of training label: (9828,)
shape of test data: (4212, 100, 200, 3)
shape of test label: (4212,)


In [ ]:
X_train_rms,Y_train_rms,X_test_rms,Y_test_rms=Load_data_all('RMS')

shape of training data: (9828, 100, 200, 3)
shape of training label: (9828,)
shape of test data: (4212, 100, 200, 3)
shape of test label: (4212,)


In [ ]:
np.save('/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/X_train_rms.npy', X_train_rms)
np.save('/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/Y_train_rms.npy', Y_train_rms)
np.save('/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/X_test_rms.npy', X_test_rms)
np.save('/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/Y_test_rms.npy', Y_test_rms)

In [ ]:
np.save('/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/X_train_mav.npy', X_train_mav)
np.save('/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/Y_train_mav.npy', Y_train_mav)
np.save('/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/X_test_mav.npy', X_test_mav)
np.save('/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/Y_test_mav.npy', Y_test_mav)

In [ ]:
np.save('/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/X_train_iemg.npy', X_train_iemg)
np.save('/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/Y_train_iemg.npy', Y_train_iemg)
np.save('/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/X_test_iemg.npy', X_test_iemg)
np.save('/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/Y_test_iemg.npy', Y_test_iemg)

In [ ]:
# np.save('/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/RMS.npy', RMS)

In [ ]:
X_train_rms_t=np.load('/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/X_train_rms.npy',allow_pickle=True)

(9828, 100, 200, 3)

In [ ]:
# X_train_rms=RMS['train_data']
# Y_train_rms=RMS['train_label']
# X_test_rms=RMS['test_data']
# Y_test_rms=RMS['test_label']
RMS['test_label']

In [ ]:
num_example=X_train_iemg.shape[0]
arr=np.arange(num_example)
np.random.shuffle(arr)
X_train_iemg=X_train_iemg[arr]
Y_train_iemg=Y_train_iemg[arr]
Y_train_iemg = to_categorical(Y_train_iemg,num_classes=53)
Y_test_iemg = to_categorical(Y_test_iemg,num_classes=53)

X_train_mav=X_train_mav[arr]
Y_train_mav=Y_train_mav[arr]
Y_train_mav = to_categorical(Y_train_mav,num_classes=53)
Y_test_mav = to_categorical(Y_test_mav,num_classes=53)

Load model

In [ ]:

IEMG=Input(shape=(100,200,3),name='IEMG')
MAV=Input(shape=(100,200,3),name='MAV')

l_iemg=Conv2D(16,3,strides=(1, 1),padding='same',activation='relu',kernel_regularizer=regularizers.l2(0.01))(IEMG)
l_mav=Conv2D(16,3,strides=(1, 1),padding='same',activation='relu',kernel_regularizer=regularizers.l2(0.01))(MAV)
l_iemg=Conv2D(16,1,strides=(1, 1),padding='same',activation='relu',kernel_regularizer=regularizers.l2(0.01))(IEMG)
l_mav=Conv2D(16,1,strides=(1, 1),padding='same',activation='relu',kernel_regularizer=regularizers.l2(0.01))(MAV)
late_fusion=concatenate([l_iemg,l_mav])
late_fusion=Dense(16,activation='relu')(late_fusion)
# late_fusion=Dropout(0.2)(late_fusion)


early_fusion=concatenate([IEMG,MAV])
early_fusion=Conv2D(32,3,strides=(1, 1),padding='same',activation='relu',kernel_regularizer=regularizers.l2(0.01))(early_fusion)
early_fusion=Conv2D(32,1,strides=(1, 1),padding='same',activation='relu',kernel_regularizer=regularizers.l2(0.01))(early_fusion)
early_fusion=Dense(16,activation='relu')(early_fusion)
# early_fusion=Dropout(0.2)(early_fusion)

mix=concatenate([early_fusion,late_fusion])
mix=Dense(3,activation='relu')(mix)
# mix=BatchNormalization()(mix)



x=Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=(100,200,3), pooling='avg')(mix)
x = Dense(1024, activation='relu')(x)
x=Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x=Dropout(0.5)(x)
predictions = Dense(53, activation='softmax')(x)

model = Model(inputs=[IEMG,MAV], outputs=predictions)

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/model.png',show_shapes=True)

In [ ]:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

In [ ]:
# for i in range(17):
#   if (i!=13):
#     model.layers[i].trainable=True
#   else:
#     model.layers[i].trainable=False


In [ ]:
_weight='/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/mav_and_iemg.h5'
opt=Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
optsgd=SGD(
    learning_rate=0.01, momentum=0.7, nesterov=False, name='SGD')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.000001)
model.compile(optimizer=optsgd, loss='categorical_crossentropy',metrics=['categorical_accuracy']) 
callbacks = [ModelCheckpoint(_weight, monitor='val_loss', save_best_only=True),learning_rate_reduction]
history = model.fit(         [X_train_iemg,X_train_mav],
                    Y_train_iemg,
                    batch_size=32,
                    epochs=32,
                    validation_split=0.1,
                    callbacks=callbacks)

In [ ]:
model.load_weights(_weight)
loss, accuracy = model.evaluate([X_test_iemg,X_test_mav],
                  Y_test_iemg)
print('loss = {0:.4f}, accuracy = {1:.4f}'.format(loss, accuracy))

In [ ]:
# for i in range(17):
#   if (i!=13):
#     model.layers[i].trainable=False
#   else:
#     model.layers[i].trainable=True


In [ ]:

# _weight='/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/mav_and_iemg.h5'
# opt=Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
# learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
# model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['categorical_accuracy']) 
# callbacks = [ModelCheckpoint(_weight, monitor='val_loss', save_best_only=True),learning_rate_reduction]
# # model.load_weights(_weight)
# history = model.fit(         [X_train_iemg,X_train_mav],
#                     Y_train_iemg,
#                     batch_size=32,
#                     epochs=48,
#                     validation_split=0.1,
#                     callbacks=callbacks)

In [ ]:
# model.load_weights(_weight)
# loss, accuracy = model.evaluate([X_test_iemg,X_test_mav],
#                   Y_test_iemg)
# print('loss = {0:.4f}, accuracy = {1:.4f}'.format(loss, accuracy))

In [ ]:
base_model=Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=(100,200,3), pooling='avg')
x=base_model.output
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(53, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
opt=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
optsgd=SGD(
    learning_rate=0.01, momentum=0.7, nesterov=False, name='SGD')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
model.compile(optimizer=optsgd, loss='categorical_crossentropy',metrics=['categorical_accuracy']) 
callbacks = [ModelCheckpoint(_weight, monitor='val_loss', save_best_only=True),learning_rate_reduction]
history = model.fit(         X_train_mav,
                    Y_train_mav,
                    epochs=64,
                    validation_split=0.1,
                    callbacks=callbacks)


In [ ]:
model.load_weights(_weight)
loss, accuracy = model.evaluate( X_test_mav,
                  Y_test_mav)
print('loss = {0:.4f}, accuracy = {1:.4f}'.format(loss, accuracy))

try new structure


In [ ]:
# base_model=Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=(100,200,3), pooling='avg')
# x=base_model.output
# x = Dense(1024, activation='relu')(x)
# x = Dense(512, activation='relu')(x)
# predictions = Dense(52, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=x)


fea1=Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=(100,200,3), pooling='avg')
for layer in fea1.layers:
    layer.name = layer.name + str('_fea1')
early_fea1=fea1.get_layer(index=55).output
# middle_fea1=fea1.get_layer(index=115).output
late_fea1=fea1.get_layer(index=125).output

fea2=Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=(100,200,3), pooling='avg')
for layer in fea2.layers:
    layer.name = layer.name + str('_fea2')
early_fea2=fea2.get_layer(index=55).output
# middle_fea2=fea2.get_layer(index=115).output
late_fea2=fea2.get_layer(index=125).output

early_fea1=Conv2D(512,3,strides=(1, 1),padding='same',activation='relu',kernel_regularizer=regularizers.l2(0.01))(early_fea1)
early_fea2=Conv2D(512,3,strides=(1, 1),padding='same',activation='relu',kernel_regularizer=regularizers.l2(0.01))(early_fea2)
early_fea1=Conv2D(128,1,strides=(1, 1),padding='same',activation='relu',kernel_regularizer=regularizers.l2(0.01))(early_fea1)
early_fea2=Conv2D(128,1,strides=(1, 1),padding='same',activation='relu',kernel_regularizer=regularizers.l2(0.01))(early_fea2)
early=concatenate([early_fea1,early_fea2])
early=GlobalAveragePooling2D()(early)
early=Dense(64)(early)

late=concatenate([late_fea1,late_fea2])
late=Conv2D(512,3,strides=(1, 1),padding='same',activation='relu',kernel_regularizer=regularizers.l2(0.01))(late)
late=Conv2D(128,1,strides=(1, 1),padding='same',activation='relu',kernel_regularizer=regularizers.l2(0.01))(late)
late=GlobalAveragePooling2D()(late)
late=Dense(64)(late)

out=concatenate([early,late])
out=Dense(53)(out)


model = Model(inputs=[fea1.input,fea2.input], outputs=out)


In [ ]:

base_model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/model.png',show_shapes=True)

In [ ]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)
